In [1]:
import numpy as np
import os
import time
from PIL import Image
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import keras
from tensorflow.keras import layers, datasets
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

In [3]:
def get_basemodel_shape(type):
    if type == 'EfficientNet-B4':
        return (380, 380, 3)
#         return (224, 224, 3)
    elif type == 'VGG16':
        return (224, 224, 3)
    elif type == 'ResNet50':
        return (224, 224, 3)

def get_base_model(type='Mobilenet'):
    if type == 'EfficientNet-B4':
        base_model = tf.keras.applications.efficientnet.EfficientNetB4(
            include_top=False,
            weights='imagenet',
            input_shape=get_basemodel_shape(type)
        )
    elif type == 'VGG16':
        base_model = tf.keras.applications.vgg16.VGG16(
            include_top=False,
            weights='imagenet',
            input_shape=get_basemodel_shape(type)
        )
    elif type == 'ResNet50':
        base_model = tf.keras.applications.resnet50.ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=get_basemodel_shape(type)
        )
    return base_model

In [4]:
# cache path
cache_path = r'D:\projects\Vision\ckpts'
if not os.path.exists(cache_path):
    os.mkdir(cache_path)
    
def get_mdl_name(data_type, modl_type, unfrozen_layers):
    if unfrozen_layers is not None:
        save_path = os.path.join(r'D:\projects\Vision\ckpts', r'%s_%s_%s' % (data_type, modl_type, unfrozen_layers))
        if not os.path.exists(save_path):
            os.mkdir(save_path)
    else:
        save_path = cache_path
    
    mdl_save_name = data_type + '_' + modl_type + '_' + \
                time.strftime("%Y%m%d", time.localtime(time.time())) + '_{epoch:02d}-{val_accuracy:.3f}.hdf5'
    abs_model_name = os.path.join(save_path, mdl_save_name)
    
    return abs_model_name

In [5]:
train_img_gen = ImageDataGenerator(fill_mode='nearest',
                                   width_shift_range = 0.15,
                                    height_shift_range = 0.15,
                                    shear_range = 0,
                                    rotation_range = 15,
                                    zoom_range = 0.1,
                                    vertical_flip=False,
                                    horizontal_flip=False
                                                 ,samplewise_center=True
                                                 ,samplewise_std_normalization=True
                                                ) 

val_img_gen = ImageDataGenerator(fill_mode='nearest'
                                                 ,samplewise_center=True
                                                 ,samplewise_std_normalization=True
                                                ) 

In [6]:
def load_data(data_type, target_size, batch_size):
    # data path
    if data_type == 'esophagus_28':
        train_data_path = r'\\192.168.0.154\数据\口咽部\大部位27分类\res_train_2'
        val_data_path = r'\\192.168.0.154\数据\口咽部\大部位27分类\res_val'
        test_data_path = r'\\192.168.0.154\数据\口咽部\大部位27分类\res_test'
    else:
        train_data_path = r'\\192.168.0.154\数据\口咽部\第8轮\res_train'
        val_data_path = r'\\192.168.0.154\数据\口咽部\第8轮\res_val'
        test_data_path = r'\\192.168.0.154\数据\口咽部\第8轮\res_test'
    #     test_data_path = '\\192.168.0.154\数据\口咽部\第8轮\valid_date_结果\富士'
    
    train_gen = train_img_gen.flow_from_directory(train_data_path,
                                    target_size = target_size,
                                    color_mode = 'rgb',
#                                     classes = [''],
                                    class_mode = 'categorical',
                                    batch_size = batch_size,
                                    shuffle = True,
                                    seed = None,
                                    interpolation = 'nearest',
                                    keep_aspect_ratio=False
                                                 )
    print('Train data loaded: total: %s, batch size: %s, num classes: %s\n' % (train_gen.n, train_gen.batch_size, train_gen.num_classes))
    
    val_gen = val_img_gen.flow_from_directory(val_data_path,
                                    target_size = target_size,
                                    color_mode = 'rgb',
#                                     classes = [''],
                                    class_mode = 'categorical',
                                    batch_size = batch_size,
                                    shuffle = False,
                                    seed = None,
                                    interpolation = 'nearest',
                                    keep_aspect_ratio=False
                                             )
    print('Val data loaded: total: %s, batch size: %s, num classes: %s\n' % (val_gen.n, val_gen.batch_size, val_gen.num_classes))
    
    return train_gen, val_gen

In [7]:
def build_model(type='Mobilenet', num_classes=27):
#     scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)

    base_model, input_shape = get_base_model(type), get_basemodel_shape(type)
    base_model.trainable = False
    inputs = keras.Input(shape=input_shape)
    
    # 模型定义
#     x = scale_layer(inputs)
    x = inputs
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # Regularize with dropout
    x = tf.keras.layers.Dense(num_classes)(x)
#     outputs = tf.keras.layers.Softmax()(x)
    outputs = x
    model = tf.keras.Model(inputs, outputs)
    
    model.summary()
    
    return model

In [8]:
# def scheduler(epoch, lr):
#     epoch = tf.cast(tf.math.add(epoch, 0), tf.float32)
#     if epoch > 10:
#         n_10 = tf.math.floordiv(epoch, 10)
#         n_10_m = tf.math.multiply(n_10, 10)
#         n_10_s = tf.math.subtract(epoch, n_10_m)
#         if n_10_s == 0:
#             lr = tf.math.multiply(tf.math.pow(0.1, n_10), 0.01)
#         else:
#             lr = lr * tf.math.exp(-0.1)
#     return lr

def scheduler(epoch, lr):
    if epoch > 2:
        lr = lr * tf.math.exp(-0.051)
    return lr
    
early_stop = EarlyStopping(monitor='val_accuracy', patience=3)
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

def get_callbacks(modl_type, data_type, unfrozen_layers=None):
    model_checkpoint = ModelCheckpoint(get_mdl_name(data_type, modl_type, unfrozen_layers), monitor='val_accuracy', verbose=2, save_best_only=True)
    callbacks = [early_stop, model_checkpoint, scheduler_callback]
    return callbacks

In [9]:
scheduler(21, 0.01)

<tf.Tensor: shape=(), dtype=float32, numpy=0.009502786>

In [10]:
modl_types = ['VGG16', 'ResNet50', 'EfficientNet-B4']
data_types = ['esophagus_28', 'pharynx_5']
batch_sizes = 4, 32
def transfer_learning(mdl_idx, data_idx, init_lr=1e-2, epochs=5, batch_size=64):
    modl_type = modl_types[mdl_idx]
    img_shape = get_basemodel_shape(modl_type)[:2]
    data_type = data_types[data_idx]
    BATCH_SIZE = batch_size
    
    # 参数
    num_clses = int(data_type.split('_')[1])

    # 定义模型
    model = build_model(type=modl_type, num_classes=num_clses)
    model.compile(optimizer = Adam(learning_rate = init_lr), 
                      loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                      metrics = ['accuracy'])

    # 加载数据
    train_gen, val_gen = load_data(data_type, img_shape, BATCH_SIZE)


    # 训练
    history = model.fit(train_gen, steps_per_epoch=train_gen.n // BATCH_SIZE, epochs=epochs,
                        validation_data=val_gen, validation_steps=val_gen.n // BATCH_SIZE,
                        callbacks=get_callbacks(modl_type, data_type), workers=1)


In [11]:
def finetune(mdl_idx, data_idx, chpkt_path, init_lr=1e-5, unfrozen_layers=-200, epochs=5, batch_size=16):
    modl_type = modl_types[mdl_idx]
    img_shape = get_basemodel_shape(modl_type)[:2]
    data_type = data_types[data_idx]
    BATCH_SIZE = batch_size
    
    # 参数
    num_clses = int(data_type.split('_')[1])
    
    # 定义模型
    model = tf.keras.models.load_model(chpkt_path)
#     model.trainable = True
    print(len(model.layers[1].layers))
    for layer in model.layers[1].layers[unfrozen_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
            
    model.compile(optimizer = Adam(learning_rate = init_lr), 
                          loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                          metrics = ['accuracy'])
    model.summary()
    
    # 加载数据
    train_gen, val_gen = load_data(data_type, img_shape, BATCH_SIZE)


    # 训练
    history = model.fit(train_gen, steps_per_epoch=train_gen.n // BATCH_SIZE, epochs=epochs,
                        validation_data=val_gen, validation_steps=val_gen.n // BATCH_SIZE,
                        callbacks=get_callbacks(modl_type, data_type, unfrozen_layers), workers=1)

In [12]:
def find_best_chkpt(mdl_idx, data_idx, unfrozen_layers):
    modl_type = modl_types[mdl_idx]
    data_type = data_types[data_idx]
    header = data_type + '_' + modl_type
    
    results = []
    if unfrozen_layers is not None:
        mdl_dir = os.path.join(cache_path, '%s_%s_%s' % (data_type, modl_type, unfrozen_layers))
        if os.path.exists(mdl_dir):
            for f in os.listdir(mdl_dir):
                if f.startswith(header):
                    acc = float(f.split('-')[-1][:-5]) * 1000
                    results.append((acc, os.path.join(mdl_dir, f)))
            results = sorted(results, key=lambda x: x[0])
            if len(results) > 0:
                print('Best ckpt Selected: %s' % results[-1][1])
                return results[-1][1]
    
    # ckpts文件夹找最大值模型
    for f in os.listdir(cache_path):
        if os.path.isfile(os.path.join(cache_path, f)) and f.startswith(header):
            print(f)
            acc = float(f.split('-')[-1][:-5]) * 1000
            results.append((acc, os.path.join(cache_path, f)))
    results = sorted(results, key=lambda x: x[0])
    
    print('Best ckpt Selected: %s' % results[-1][1])
    return results[-1][1]

In [13]:
def run(mdl_idx, data_idx):
    transfer_learning(mdl_idx, data_idx, 1e-2, 3, 16)
    finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-5, -475, 50, 8)
    finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-6, -475, 20, 8)
#     finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-7, 5)

In [15]:
mdl_idx, data_idx = 2, 0
# transfer_learning(mdl_idx, data_idx, 1e-2, 3, 16)
# finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-5, 120, 50, 4)
finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-6, 120, 20, 4)

# run(1, 1)
# finetune(1, 1, find_best_chkpt(1, 1), 1e-7)
# finetune(0, 1, r'D:\projects\Vision\ckpts\pharynx_5_VGG16_20220527_10-0.953.hdf5', 1e-2)

Best ckpt Selected: esophagus_28_EfficientNet-B4_20220601_240-0.891.hdf5
475
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 7, 7, 1792)       17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 28)                50204     
                                                                 
Total params: 17,724,027
Trainable params: 17,473,

In [17]:
mdl_idx, data_idx = 2, 0
finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-5, 360, 5, 4)

Best ckpt Selected: esophagus_28_EfficientNet-B4_20220617_Finetune-0.241.hdf5
475
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 12, 12, 1792)     17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 28)                50204     
                                                                 
Total params: 17,724,027
Trainable params: 11

In [14]:
mdl_idx, data_idx, uf_layer = 2, 0, 240
finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx, uf_layer), 1e-5, uf_layer, 20, 4)

Best ckpt Selected: D:\projects\Vision\ckpts\esophagus_28_EfficientNet-B4_240\esophagus_28_EfficientNet-B4_20220617_05-0.863.hdf5
475
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 12, 12, 1792)     17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 28)                50204     
                                                           

In [15]:
finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx, uf_layer), 1e-6, uf_layer, 20, 4)

Best ckpt Selected: D:\projects\Vision\ckpts\esophagus_28_EfficientNet-B4_240\esophagus_28_EfficientNet-B4_20220620_01-0.864.hdf5
475
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 12, 12, 1792)     17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 28)                50204     
                                                           

In [18]:
mdl_idx, data_idx = 2, 0
for uf_layer in [80, 160]:
    finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx, uf_layer), 1e-5, uf_layer, 30, 4)
    finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx, uf_layer), 1e-6, uf_layer, 30, 4)

esophagus_28_EfficientNet-B4_20220617_TransferBase-0.241.hdf5
Best ckpt Selected: D:\projects\Vision\ckpts\esophagus_28_EfficientNet-B4_20220617_TransferBase-0.241.hdf5
475
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 12, 12, 1792)     17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 28)                50204     
                    

                                                                 
Total params: 17,724,027
Trainable params: 17,181,380
Non-trainable params: 542,647
_________________________________________________________________
Found 70719 images belonging to 28 classes.
Train data loaded: total: 70719, batch size: 4, num classes: 28

Found 7395 images belonging to 28 classes.
Val data loaded: total: 7395, batch size: 4, num classes: 28

Epoch 1/30
17679/17679 [==============================] - ETA: 0s - loss: 0.2777 - accuracy: 0.9006
Epoch 1: val_accuracy improved from -inf to 0.88420, saving model to D:\projects\Vision\ckpts\esophagus_28_EfficientNet-B4_160\esophagus_28_EfficientNet-B4_20220621_01-0.884.hdf5
17679/17679 [==============================] - 2602s 147ms/step - loss: 0.2777 - accuracy: 0.9006 - val_loss: 0.3673 - val_accuracy: 0.8842 - lr: 1.0000e-06
Epoch 2/30
17679/17679 [==============================] - ETA: 0s - loss: 0.2645 - accuracy: 0.9066
Epoch 2: val_accuracy improved fro

In [19]:
mdl_idx, data_idx = 2, 0
for uf_layer in [0]:
    finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx, uf_layer), 1e-5, uf_layer, 30, 4)
    finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx, uf_layer), 1e-6, uf_layer, 30, 4)

esophagus_28_EfficientNet-B4_20220617_TransferBase-0.241.hdf5
Best ckpt Selected: D:\projects\Vision\ckpts\esophagus_28_EfficientNet-B4_20220617_TransferBase-0.241.hdf5
475
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 12, 12, 1792)     17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 28)                50204     
                    

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  OSError: [Errno 22] Invalid argument: '\\\\192.168.0.154\\数据\\口咽部\\大部位27分类\\res_train_2\\15\\15-010001_0_001_2018_11_08_11_30_261_0_c81.jpg'
Traceback (most recent call last):

  File "D:\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)

  File "D:\Anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "D:\Anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1030, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "D:\Anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 831, in wrapped_generator
    for data in generator_fn():

  File "D:\Anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 957, in generator_fn
    yield x[i]

  File "D:\Anaconda3\lib\site-packages\keras\preprocessing\image.

In [20]:
mdl_idx, data_idx = 2, 0
for uf_layer in [0]:
    finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx, uf_layer), 1e-6, uf_layer, 30, 4)

Best ckpt Selected: D:\projects\Vision\ckpts\esophagus_28_EfficientNet-B4_0\esophagus_28_EfficientNet-B4_20220621_05-0.892.hdf5
475
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 12, 12, 1792)     17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 28)                50204     
                                                             